In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True ,reshape=False)
X_train, y_train = mnist.train.images, mnist.train.labels

assert(len(X_train) == len(y_train))

print()
print("Image Shape: {}".format(X_train[0].shape))
print("Image Shape: {}".format(y_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))

In [1]:
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))
print("Updated Image Shape: {}".format(y_train[0].shape))

NameError: name 'np' is not defined

In [5]:
X = tf.placeholder(tf.float32, [None, 32, 32, 1])
Y = tf.placeholder(tf.int32, [None,10])
is_training = tf.placeholder(tf.bool)

#LeNet
conv1=tf.layers.conv2d(X, 6, [5, 5], activation=tf.nn.relu)
conv1=tf.layers.max_pooling2d(conv1, [2, 2], [2, 2])
conv1 = tf.layers.dropout(conv1, 0.7, is_training)

conv2=tf.layers.conv2d(conv1, 16, [5, 5], activation=tf.nn.relu)
conv2=tf.layers.max_pooling2d(conv2, [2, 2], [2, 2])
conv2 = tf.layers.dropout(conv2, 0.7, is_training)

fc1 = tf.contrib.layers.flatten(conv2)
fc1 = tf.layers.dense(fc1, 120, activation=tf.nn.relu)
fc1 = tf.layers.dropout(fc1, 0.5, is_training)

fc2 = tf.layers.dense(fc1, 84, activation=tf.nn.relu)
fc2 = tf.layers.dropout(fc2, 0.5, is_training)

logits = tf.layers.dense(fc2, 10, activation=None)

In [ ]:
def train():
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

    sess.run(tf.global_variables_initializer())
    
    batch_size = 128
    epoch = 10

    for epoch in range(epoch):
        
        total_cost = 0
        
        for i in range(0, len(X_train), batch_size):
            end = i + batch_size
            batch_xs, batch_ys = X_train[i:end], y_train[i:end]
            _, cost_val = sess.run([optimizer, cost],feed_dict={X: batch_xs, Y: batch_ys, is_training: True})
            total_cost += cost_val
        print('Epoch:', '%04d' % (epoch + 1),'Avg. cost =', '{:.4f}'.format(total_cost / (len(X_train)/batch_size)))
    print('최적화 완료!')

In [ ]:
def save_variables():

    train()
    saver = tf.train.Saver()
    saver.save(sess, 'weights/test.ckpt')

In [ ]:
def restore_variables():
    
    saver = tf.train.Saver()
    saver.restore(sess, 'weights/test.ckpt')
    
    index = random.randint(0, len(X_train))
    sample_image = X_train[index].squeeze()
    
    plt.imshow(sample_image, cmap='Greys')
    sample_image = sample_image.reshape(-1, 32, 32, 1)
    print (sess.run(tf.argmax(logits, 1), feed_dict = { X : sample_image}))

In [ ]:
with tf.Session() as sess:

    if __name__ == '__main__' :
        save_variables()
        restore_variables()